In [1]:
import math
from itertools import product
from pathlib import Path
import ee
import pandas as pd
import ast

ee.Authenticate()
ee.Initialize(project='raman-461708')

"""
    Helper functions to Download High Resolution Images
"""
# Function to convert latitude to pixel Y at a given zoom level
def lat_to_pixel_y(lat, zoom):
    sin_lat = math.sin(math.radians(lat))
    pixel_y = ((0.5 - math.log((1 + sin_lat) / (1 - sin_lat)) / (4 * math.pi)) * (2 ** (zoom + 8)))
    return pixel_y

# Function to convert longitude to pixel X at a given zoom level
def lon_to_pixel_x(lon, zoom):
    pixel_x = ((lon + 180) / 360) * (2 ** (zoom + 8))
    return pixel_x

# Function to convert pixel X to longitude
def pixel_x_to_lon(pixel_x, zoom):
    lon = (pixel_x / (2 ** (zoom + 8))) * 360 - 180
    return lon

# Function to convert pixel Y to latitude
def pixel_y_to_lat(pixel_y, zoom):
    n = math.pi - 2 * math.pi * pixel_y / (2 ** (zoom + 8))
    lat = math.degrees(math.atan(math.sinh(n)))
    return lat

def lat_lon_from_pixel(lat, lon, zoom, scale):
    """
    Given a starting latitude and longitude, calculate the latitude and longitude
    of the opposite corner of a 256x256 image at a given zoom level.
    """
    pixel_x = lon_to_pixel_x(lon, zoom)
    pixel_y = lat_to_pixel_y(lat, zoom)
    
    new_lon = pixel_x_to_lon(pixel_x + 256*scale, zoom)
    new_lat = pixel_y_to_lat(pixel_y + 256*scale, zoom)

    return new_lat, new_lon

"""

Helper function for dividing an roi into blocks

"""

def get_n_boxes(lat, lon, n, zoom, scale):
    diagonal_lat_lon = [(lat, lon),]
    for i in range(n):
        new_lat_lon = lat_lon_from_pixel(lat, lon, zoom, scale)
        diagonal_lat_lon.append(new_lat_lon)
        lat, lon = new_lat_lon
    lats = [i[0] for i in diagonal_lat_lon]
    longs = [i[1] for i in diagonal_lat_lon]
    first_points = list(product(lats, longs))
    boxes = []
    for point in first_points:
        lat, lon = point
        new_lat, new_lon = lat_lon_from_pixel(lat, lon, zoom, scale)
        boxes.append([(lat, lon), (lat, new_lon), (new_lat, new_lon), (new_lat, lon)])
    return boxes

def latlon_to_tile_xy(lat, lon, zoom):
    """Converts lat/lon to tile x/y at given zoom level"""
    lat_rad = math.radians(lat)
    n = 2.0 ** zoom
    tile_x = int((lon + 180.0) / 360.0 * n)
    tile_y = int((1.0 - math.log(math.tan(lat_rad) + 1 / math.cos(lat_rad)) / math.pi) / 2.0 * n)
    return tile_x, tile_y

def tile_xy_to_latlon(tile_x, tile_y, zoom):
    """Converts top-left corner of tile x/y at given zoom level to lat/lon"""
    n = 2.0 ** zoom
    lon_deg = tile_x / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * tile_y / n)))
    lat_deg = math.degrees(lat_rad)
    return lat_deg, lon_deg

def get_csv(points):
    first_points = [i[0] for i in points]
    all_points = []
    for index, point in enumerate(first_points):
        four_first_point = get_n_boxes(point[0], point[1], 1, 17, 16)
        four_first_point = [(i[0], i[2]) for i in four_first_point]
        for top_left, bottom_right in four_first_point:
            all_points.append((index, (top_left,bottom_right)))
    return all_points

def get_points(roi):
    zoom = 17
    scale = 32
    bounds = roi.bounds().coordinates().get(0).getInfo()
    lons = sorted([i[0] for i in bounds])
    lats = sorted([i[1] for i in bounds])
    
    tile_x, tile_y = latlon_to_tile_xy(lats[-1], lons[0], zoom)
    top_left_lat, top_left_lon = tile_xy_to_latlon(tile_x, tile_y, zoom)
    
    starting_point = top_left_lat, top_left_lon
    min_, max_ = (
        [lon_to_pixel_x(top_left_lon, zoom), lat_to_pixel_y(lats[0], zoom) ],
        [lon_to_pixel_x(lons[-1], zoom), lat_to_pixel_y(top_left_lat, zoom)]
        )
    iterations = math.ceil(max(abs(min_[0] -  max_[0]), abs(min_[1] - max_[1]))/256/scale)
    points = get_n_boxes(starting_point[0], starting_point[1], iterations, zoom, scale)
    csv_points = get_csv(points)
    return points, csv_points

In [2]:
roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", 7))
#roi = ee.FeatureCollection("projects/raman-461708/assets/filtered_grids_1").limit(1)
squares, csv_points = get_points(roi)

In [3]:
df = pd.DataFrame(csv_points, columns=["index", "points"])
df.to_csv("points.csv", index=False)

In [9]:
len(squares)

9

In [10]:
roi.getInfo()

{'type': 'FeatureCollection',
 'columns': {'grid_id': 'Integer', 'system:index': 'String'},
 'version': 1753620561931510.0,
 'id': 'projects/raman-461708/assets/filtered_grids_1',
 'properties': {'system:asset_size': 14646},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[78.81866457168141, 18.906286538114937],
      [78.90655516333197, 18.906286521502466],
      [78.90655516333197, 18.989414745963032],
      [78.81866457168141, 18.989414682748105],
      [78.81866457168141, 18.906286538114937]]]},
   'id': '00000000000000000000',
   'properties': {'grid_id': 86}}]}

In [23]:
features = []
for i, square in enumerate(squares):
    coords = [[lon, lat] for lat, lon in square]  # Convert to [lon, lat]
    #coords.append(coords[0])  # Close the polygon
    polygon = ee.Geometry.Polygon([coords])
    feature = ee.Feature(polygon, {'grid_id': i})
    features.append(feature)

# Create FeatureCollection
fc = ee.FeatureCollection(features)

In [24]:
fc.filterBounds(roi).size().getInfo()

1984

In [25]:
task = ee.batch.Export.table.toAsset(
    collection=fc.filterBounds(roi),
    description='exportToTableAssetExample',
    assetId='projects/raman-461708/assets/telangana_aez',
)
task.start()

In [6]:
fc.filterBounds(roi).size().getInfo()

1984